<a href="https://colab.research.google.com/github/williammcintosh/ChequeWriterApp/blob/main/BookingTimes__SQL_Challenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About

- Author - Will McIntosh
- For BookingTimes' Technical Challenge

## Results

- For the results, drop all the way to the bottom!

## Explanation of Code

### Indexing

- **`client_id` Primary Key**:
  - Automatically indexed, ensuring unique identification of each client and efficient retrieval.
- **Index on `client_name`**:
  - `CREATE INDEX idx_client_name ON Client(client_name);`
  - Speeds up search queries involving client names, especially when checking for duplicates or sorting.

### Naming Conventions

- **Table Names**:
  - **`Client`** and **`Booking`**: Capitalized and singular form, enhancing clarity and readability.
- **Column Names**:
  - **`client_id`, `client_name`, `client_email`, `client_phone`**:
    - Use `snake_case` for consistency and readability.
    - Descriptive, indicating the data stored in each column.
  - **`booking_id`, `booking_date`, `booking_amount`**:
    - Follow the same naming pattern, making the schema intuitive and easy to maintain.

### Referential Integrity

- **No Foreign Key Constraints**:
  - Since Adam mentioned that he doesn't use foreign keys in BookingTimes I thought I would use the same logic when implementing that in my smaller databases.
  - The `client_id` in the `Booking` table references the `client_id` in the `Client` table, but no foreign key constraints are enforced.
  - This setup allows more flexibility in the data structure but requires manual handling to ensure that each booking corresponds to a valid client.

### Additional Benefits

- **Unique Constraint on `client_email`**:
  - Prevents duplicate entries for emails, ensuring data accuracy.
- **Scalable Design**:
  - The schema supports easy extension for new tables and relationships, following best practices for database normalization.


In [ ]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Connect to SQLite database (or create it)
conn = sqlite3.connect('clients_bookings.db')

# Create a cursor object using the cursor method
cursor = conn.cursor()

# Drop tables if they already exist to ensure a clean slate
cursor.execute('DROP TABLE IF EXISTS Client')
cursor.execute('DROP TABLE IF EXISTS Booking')

# Create the Client table with improved naming and indexing
create_client_table_query = '''
CREATE TABLE Client (
    client_id INTEGER PRIMARY KEY AUTOINCREMENT,  -- Primary key with descriptive name
    client_name TEXT NOT NULL,
    client_email TEXT UNIQUE,  -- Enforce unique email constraint
    client_phone TEXT
);
'''
cursor.execute(create_client_table_query)

# Create an index on client_name for better search performance
cursor.execute('CREATE INDEX idx_client_name ON Client(client_name);')

# Insert data into the Client table, including some duplicate names
clients_data = [
    ('John Doe', 'john@example.com', '123-456-7890'),
    ('Jane Smith', 'jane@example.com', '234-567-8901'),
    ('John Doe', 'john.doe2@example.com', '345-678-9012'),  # Duplicate name
    ('Alice Johnson', 'alice@example.com', '456-789-0123'),
    ('Bob Brown', 'bob@example.com', '567-890-1234'),
    ('Jane Smith', 'jane.smith2@example.com', '678-901-2345'),  # Duplicate name
    ('Michael White', 'michael@example.com', '789-012-3456'),
    ('Alice Johnson', 'alice.j2@example.com', '890-123-4567')   # Duplicate name
]

# Insert data into the Client table
cursor.executemany('INSERT INTO Client (client_name, client_email, client_phone) VALUES (?, ?, ?)', clients_data)

# Commit the changes
conn.commit()

# Create the Booking table without foreign key constraints
create_booking_table_query = '''
CREATE TABLE Booking (
    booking_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,  -- Reference to client_id without foreign key constraint
    booking_date TEXT,
    booking_amount REAL
);
'''
cursor.execute(create_booking_table_query)

# Insert sample data into the Booking table
bookings_data = [
    (1, '2024-08-01', 250.75),  # Booking for John Doe
    (2, '2024-08-02', 100.50),  # Booking for Jane Smith
    (1, '2024-08-03', 300.00),  # Another booking for John Doe
    (4, '2024-08-04', 175.25),  # Booking for Alice Johnson
    (5, '2024-08-05', 220.40),  # Booking for Bob Brown
    (2, '2024-08-06', 400.60)   # Another booking for Jane Smith
]

# Insert data into the Booking table
cursor.executemany('INSERT INTO Booking (client_id, booking_date, booking_amount) VALUES (?, ?, ?)', bookings_data)

# Commit the changes
conn.commit()

# Query to find records with duplicate names in the Client table
query_duplicates = '''
SELECT client_name, COUNT(*) as count
FROM Client
GROUP BY client_name
HAVING COUNT(*) > 1;
'''

# Execute the query and fetch the results
cursor.execute(query_duplicates)
duplicates = cursor.fetchall()

# Display the duplicate clients
duplicates_df = pd.DataFrame(duplicates, columns=['Name', 'Count'])
print("Duplicate Clients:")
print(duplicates_df)

# Query to display bookings with client information
query_bookings = '''
SELECT b.booking_id, c.client_name, b.booking_date, b.booking_amount
FROM Booking b
JOIN Client c ON b.client_id = c.client_id;
'''

# Execute the query and fetch the results
cursor.execute(query_bookings)
bookings = cursor.fetchall()

# Display the bookings with client information
bookings_df = pd.DataFrame(bookings, columns=['Booking ID', 'Client Name', 'Booking Date', 'Booking Amount'])
print("\nBookings with Client Information:")
print(bookings_df)

# Close the connection
conn.close()


Duplicate Clients:
            Name  Count
0  Alice Johnson      2
1     Jane Smith      2
2       John Doe      2

Orders with Client Information:
   Order ID    Client Name  Order Date  Order Amount
0         1       John Doe  2024-08-01        250.75
1         2     Jane Smith  2024-08-02        100.50
2         3       John Doe  2024-08-03        300.00
3         4  Alice Johnson  2024-08-04        175.25
4         5      Bob Brown  2024-08-05        220.40
5         6     Jane Smith  2024-08-06        400.60
